In [5]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge


  0%|          | 0.00/285M [00:00<?, ?B/s]
  0%|          | 1.00M/285M [00:00<01:01, 4.82MB/s]
  1%|          | 2.00M/285M [00:00<01:06, 4.47MB/s]
  1%|1         | 3.00M/285M [00:00<00:52, 5.68MB/s]
  1%|1         | 4.00M/285M [00:00<00:45, 6.51MB/s]
  2%|1         | 5.00M/285M [00:01<02:23, 2.05MB/s]
  2%|2         | 6.00M/285M [00:02<02:01, 2.42MB/s]
  2%|2         | 7.00M/285M [00:02<01:39, 2.92MB/s]
  3%|2         | 8.00M/285M [00:02<01:21, 3.56MB/s]
  3%|3         | 9.00M/285M [00:02<01:12, 4.02MB/s]
  4%|3         | 10.0M/285M [00:03<01:30, 3.17MB/s]
  4%|3         | 11.0M/285M [00:03<01:20, 3.59MB/s]
  4%|4         | 12.0M/285M [00:03<01:12, 3.96MB/s]
  5%|4         | 13.0M/285M [00:03<01:05, 4.34MB/s]
  5%|4         | 14.0M/285M [00:03<00:57, 4.97MB/s]
  5%|5         | 15.0M/285M [00:04<01:02, 4.57MB/s]
  6%|5         | 16.0M/285M [00:04<00:53, 5.28MB/s]
  6%|5         | 17.0M/285M [00:04<00:54, 5.13MB/s]
  6%|6         | 18.0M/285M [00:04<00:53, 5.21MB/s]
  7%|6         | 19.

 65%|######4   | 185M/285M [00:53<00:18, 5.72MB/s]
 65%|######5   | 186M/285M [00:53<00:16, 6.15MB/s]
 66%|######5   | 187M/285M [00:53<00:16, 6.31MB/s]
 66%|######5   | 188M/285M [00:54<00:28, 3.61MB/s]
 66%|######6   | 189M/285M [00:54<00:23, 4.24MB/s]
 67%|######6   | 190M/285M [00:54<00:20, 4.76MB/s]
 67%|######6   | 191M/285M [00:54<00:19, 5.05MB/s]
 67%|######7   | 192M/285M [00:55<00:28, 3.46MB/s]
 68%|######7   | 193M/285M [00:55<00:26, 3.59MB/s]
 68%|######8   | 194M/285M [00:56<00:25, 3.82MB/s]
 68%|######8   | 195M/285M [00:56<00:24, 3.92MB/s]
 69%|######8   | 196M/285M [00:56<00:23, 3.91MB/s]
 69%|######9   | 197M/285M [00:57<00:29, 3.17MB/s]
 69%|######9   | 198M/285M [00:57<00:27, 3.32MB/s]
 70%|######9   | 199M/285M [00:57<00:25, 3.51MB/s]
 70%|#######   | 200M/285M [00:57<00:24, 3.61MB/s]
 70%|#######   | 201M/285M [00:58<00:21, 4.03MB/s]
 71%|#######   | 202M/285M [00:58<00:21, 4.01MB/s]
 71%|#######1  | 203M/285M [00:58<00:23, 3.74MB/s]
 72%|#######1  | 204M/285M [00:

In [6]:
!unzip -qq challenges-in-representation-learning-facial-expression-recognition-challenge.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!pip install kaggle

In [2]:
  
import csv
import glob
import pathlib

import cv2
import imutils
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.utils import to_categorical

EMOTIONS = ['angry', 'scared', 'happy', 'sad', 'surprised',
            'neutral']

COLORS = {
    'angry': (0, 0, 255),
    'scared': (0, 128, 255),
    'happy': (0, 255, 255),
    'sad': (255, 0, 0),
    'surprised': (178, 255, 102),
    'neutral': (160, 160, 160)
}


def build_network(input_shape, classes):
    input = Input(shape=input_shape)
    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               padding='same',
               kernel_initializer='he_normal')(input)
    x = ELU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               kernel_initializer='he_normal',
               padding='same')(x)
    x = ELU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.25)(x)

    x = Conv2D(filters=64,
               kernel_size=(3, 3),
               kernel_initializer='he_normal',
               padding='same')(x)
    x = ELU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=64,
               kernel_size=(3, 3),
               kernel_initializer='he_normal',
               padding='same')(x)
    x = ELU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.25)(x)

    x = Conv2D(filters=128,
               kernel_size=(3, 3),
               kernel_initializer='he_normal',
               padding='same')(x)
    x = ELU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=128,
               kernel_size=(3, 3),
               kernel_initializer='he_normal',
               padding='same')(x)
    x = ELU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.25)(x)

    x = Flatten()(x)
    x = Dense(units=64,
              kernel_initializer='he_normal')(x)
    x = ELU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(rate=0.5)(x)

    x = Dense(units=64,
              kernel_initializer='he_normal')(x)
    x = ELU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(rate=0.5)(x)

    x = Dense(units=classes,
              kernel_initializer='he_normal')(x)
    output = Softmax()(x)

    return Model(input, output)


def load_dataset(dataset_path, classes):
    train_images = []
    train_labels = []
    val_images = []
    val_labels = []
    test_images = []
    test_labels = []

    with open(dataset_path, 'r') as f:
        reader = csv.DictReader(f)
        for line in reader:
            label = int(line['emotion'])

            if label <= 1:
                label = 0  # This merges classes 1 and 0.

            if label > 0:
                label -= 1  # All classes start from 0.

            image = np.array(line['pixels'].split(' '),
                             dtype='uint8')
            image = image.reshape((48, 48))
            image = img_to_array(image)

            if line['Usage'] == 'Training':
                train_images.append(image)
                train_labels.append(label)
            elif line['Usage'] == 'PrivateTest':
                val_images.append(image)
                val_labels.append(label)
            else:
                test_images.append(image)
                test_labels.append(label)

    train_images = np.array(train_images)
    val_images = np.array(val_images)
    test_images = np.array(test_images)

    train_labels = to_categorical(np.array(train_labels),
                                  classes)
    val_labels = to_categorical(np.array(val_labels), classes)
    test_labels = to_categorical(np.array(test_labels),
                                 classes)

    return (train_images, train_labels), \
           (val_images, val_labels), \
           (test_images, test_labels)


def rectangle_area(r):
    return (r[2] - r[0]) * (r[3] - r[1])


def plot_emotion(emotions_plot, emotion, probability, index):
    w = int(probability * emotions_plot.shape[1])
    cv2.rectangle(emotions_plot,
                  (5, (index * 35) + 5),
                  (w, (index * 35) + 35),
                  color=COLORS[emotion],
                  thickness=-1)

    white = (255, 255, 255)
    text = f'{emotion}: {probability * 100:.2f}%'
    cv2.putText(emotions_plot,
                text,
                (10, (index * 35) + 23),
                fontFace=cv2.FONT_HERSHEY_COMPLEX,
                fontScale=0.45,
                color=white,
                thickness=2)

    return emotions_plot


def plot_face(image, emotion, detection):
    frame_x, frame_y, frame_width, frame_height = detection
    
   cv2.rectangle(image,
                  (frame_x, frame_y),
                  (frame_x + frame_width,
                   frame_y + frame_height),
                  color=COLORS[emotion],
                  thickness=2)
    cv2.putText(image,
                emotion,
                (frame_x, frame_y - 10),
                fontFace=cv2.FONT_HERSHEY_COMPLEX,
                fontScale=0.45,
                color=COLORS[emotion],
                thickness=2)

    return image


def predict_emotion(model, roi):
    roi = cv2.resize(roi, (48, 48))
    roi = roi.astype('float') / 255.0
    roi = img_to_array(roi)
    roi = np.expand_dims(roi, axis=0)

    predictions = model.predict(roi)[0]
    return predictions


def run_model(EPOCHS=2):
    base_path = (pathlib.Path.home() / 'dummy_folder')
    input_path = str(base_path / 'fer2013.csv')
    classes = len(EMOTIONS)

    (train_images, train_labels), \
    (val_images, val_labels), \
    (test_images, test_labels) = load_dataset(input_path,
                                              classes)

    model = build_network((48, 48, 1), classes)
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=0.003),
                  metrics=['accuracy'])

    checkpoint_pattern = ('model-ep{epoch:03d}-loss{loss:.3f}'
                          '-val_loss{val_loss:.3f}.h5')
    checkpoint = ModelCheckpoint(checkpoint_pattern,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='min')

    BATCH_SIZE = 128
    train_augmenter = ImageDataGenerator(rotation_range=10,
                                         zoom_range=0.1,
                                         horizontal_flip=True,
                                         rescale=1. / 255.,
                                         fill_mode='nearest')
    train_gen = train_augmenter.flow(train_images,
                                     train_labels,
                                     batch_size=BATCH_SIZE)
    train_steps = len(train_images) // BATCH_SIZE

    val_augmenter = ImageDataGenerator(rescale=1. / 255.)
    val_gen = val_augmenter.flow(val_images,
                                 val_labels,
                                 batch_size=BATCH_SIZE)

    
    model.fit(train_gen,
              steps_per_epoch=train_steps,
              validation_data=val_gen,
              epochs=EPOCHS,
              verbose=1,
              callbacks=[checkpoint])

    test_augmenter = ImageDataGenerator(rescale=1. / 255.)
    test_gen = test_augmenter.flow(test_images,
                                   test_labels,
                                   batch_size=BATCH_SIZE)
    test_steps = len(test_images) // BATCH_SIZE
    _, accuracy = model.evaluate(test_gen, steps=test_steps)

    print(f'Accuracy: {accuracy * 100}%')
 

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 172)

In [12]:
run_model()

C:\Users\venka\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/2
224/224 [==============================] - ETA: 0s - loss: 1.9076 - accuracy: 0.2728
Epoch 00001: val_loss improved from inf to 1.52059, saving model to model-ep001-loss1.908-val_loss1.521.h5
224/224 [==============================] - 336s 1s/step - loss: 1.9076 - accuracy: 0.2728 - val_loss: 1.5206 - val_accuracy: 0.3826
Epoch 2/2
224/224 [==============================] - ETA: 0s - loss: 1.4857 - accuracy: 0.4054
Epoch 00002: val_loss improved from 1.52059 to 1.30789, saving model to model-ep002-loss1.486-val_loss1.308.h5
28/28 [==============================] - 13s 461ms/step - loss: 1.3159 - accuracy: 0.4883
Accuracy: 48.828125%


In [36]:
import keras

model = keras.models.load_model("dummy_folder/model-ep061-loss0.795-val_loss0.882.h5")

video_path = 'dummy_folder/emotions.mp4'
# video_path = '0'
camera = cv2.VideoCapture(video_path)  # Pass 0 to use webcam

det = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

#cascade_file = 'resources/haarcascade_frontalface_default.xml'
#det = cv2.CascadeClassifier(cascade_file)
while True:
    frame_exists, frame = camera.read()

    print(frame)
    if not frame_exists:
        break

    frame = imutils.resize(frame, width=380)
    emotions_plot = np.zeros_like(frame, dtype='uint8')
    copy = frame.copy()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    detections = det.detectMultiScale(gray,
                             scaleFactor=1.1,
                             minNeighbors=5,
                             minSize=(35, 35),
                             flags=cv2.CASCADE_SCALE_IMAGE)
    

    if len(detections) > 0:
        detections = sorted(detections,
                            key=rectangle_area)
        best_detection = detections[-1]

        (frame_x, frame_y,
         frame_width, frame_height) = best_detection
        
        roi = gray[frame_y:frame_y + frame_height,
                   frame_x:frame_x + frame_width]
        predictions = predict_emotion(model, roi)
        label = EMOTIONS[predictions.argmax()]

        for i, (emotion, probability) in \
                enumerate(zip(EMOTIONS, predictions)):
            emotions_plot = plot_emotion(emotions_plot,
                                         emotion,
                                         probability,
                                         i)

        clone = plot_face(copy, label, best_detection)

    cv2.imshow('Face & emotions',
               np.hstack([copy, emotions_plot]))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

[[[17 17 17]
  [17 17 17]
  [17 17 17]
  ...
  [15 15 15]
  [15 15 15]
  [13 13 13]]

 [[17 17 17]
  [17 17 17]
  [17 17 17]
  ...
  [15 15 15]
  [15 15 15]
  [13 13 13]]

 [[17 17 17]
  [17 17 17]
  [17 17 17]
  ...
  [15 15 15]
  [15 15 15]
  [13 13 13]]

 ...

 [[19 19 19]
  [19 19 19]
  [20 20 20]
  ...
  [ 1  4  9]
  [ 0  3  8]
  [ 0  0  5]]

 [[19 19 19]
  [20 20 20]
  [20 20 20]
  ...
  [ 0  4  6]
  [ 3  6 11]
  [ 0  3  8]]

 [[19 19 19]
  [20 20 20]
  [20 20 20]
  ...
  [ 0  3  5]
  [ 4  7 12]
  [ 3  6 11]]]
[[170  38 160 160]]
[[[17 17 17]
  [17 17 17]
  [17 17 17]
  ...
  [15 15 15]
  [15 15 15]
  [13 13 13]]

 [[17 17 17]
  [17 17 17]
  [17 17 17]
  ...
  [15 15 15]
  [15 15 15]
  [13 13 13]]

 [[17 17 17]
  [17 17 17]
  [17 17 17]
  ...
  [15 15 15]
  [15 15 15]
  [13 13 13]]

 ...

 [[19 19 19]
  [19 19 19]
  [20 20 20]
  ...
  [ 3  6 11]
  [ 0  3  8]
  [ 0  0  5]]

 [[19 19 19]
  [20 20 20]
  [20 20 20]
  ...
  [ 1  4  9]
  [ 3  6 11]
  [ 0  2  7]]

 [[19 19 19]
  [20 20 

[[[17 17 17]
  [17 17 17]
  [17 17 17]
  ...
  [15 15 15]
  [15 15 15]
  [13 13 13]]

 [[17 17 17]
  [17 17 17]
  [17 17 17]
  ...
  [15 15 15]
  [15 15 15]
  [13 13 13]]

 [[17 17 17]
  [17 17 17]
  [17 17 17]
  ...
  [15 15 15]
  [15 15 15]
  [13 13 13]]

 ...

 [[19 19 19]
  [19 19 19]
  [20 20 20]
  ...
  [56 59 64]
  [37 38 43]
  [ 1  2  7]]

 [[19 19 19]
  [20 20 20]
  [20 20 20]
  ...
  [57 60 65]
  [39 42 47]
  [ 0  1  6]]

 [[19 19 19]
  [20 20 20]
  [20 20 20]
  ...
  [55 58 63]
  [45 48 53]
  [ 0  0  4]]]
[[175  39 155 155]]
[[[17 17 17]
  [17 17 17]
  [17 17 17]
  ...
  [15 15 15]
  [15 15 15]
  [13 13 13]]

 [[17 17 17]
  [17 17 17]
  [17 17 17]
  ...
  [15 15 15]
  [15 15 15]
  [13 13 13]]

 [[17 17 17]
  [17 17 17]
  [17 17 17]
  ...
  [15 15 15]
  [15 15 15]
  [13 13 13]]

 ...

 [[19 19 19]
  [19 19 19]
  [20 20 20]
  ...
  [29 32 37]
  [69 72 77]
  [53 56 61]]

 [[19 19 19]
  [20 20 20]
  [20 20 20]
  ...
  [27 30 35]
  [71 74 79]
  [60 63 68]]

 [[19 19 19]
  [20 20 

In [ ]:
from autokeras import *